In [163]:
import pandas as pd
import numpy as np
import re
from re import search
import codecs
import sys
import string
import math
import heapq


In [212]:
stop_words = []
prefixes = []
postfixes = []
verb_roots = []
common_words = []
plural_singular = []


def read_dataset(path, name):
    df = pd.read_excel(path + name)
    return df

def read_files():
    path = 'files/'

    #stop_words
    name = 'stop_words.txt'
    f = open(path + name, 'r', encoding='utf-8')
    Lines = f.read().splitlines()
    # Strips the newline character
    for line in Lines:
        stop_words.append(line)
    f.close()

    #prefixes
    name = 'prefixes.txt'
    f = open(path + name, 'r', encoding='utf-8')
    Lines = f.read().splitlines()
    # Strips the newline character
    for line in Lines:
        prefixes.append(line)
    f.close()

    #postfixes
    name = 'postfixes.txt'
    f = open(path + name, 'r', encoding='utf-8')
    Lines = f.read().splitlines()
    # Strips the newline character
    for line in Lines:
        postfixes.append(line)
    f.close()

    # verb_roots
    name = 'verb.txt'
    f = open(path + name, 'r', encoding='utf-8')
    Lines = f.read().splitlines()
    count = 0
    # Strips the newline character
    for line in Lines:
        line = line.split('-')
        verb_roots.append(line)
    f.close()

    #common_words
    name = 'common_words.txt'
    f = open(path + name, 'r', encoding='utf-8')
    Lines = f.read().splitlines()
    # Strips the newline character
    for line in Lines:
        common_words.append(line)
    f.close()

    #plural_singular
    name = 'plural_singular.txt'
    f = open(path + name, 'r', encoding='utf-8')
    Lines = f.read().splitlines()
    count = 0
    # Strips the newline character
    for line in Lines:
        line = line.split('-')
        plural_singular.append(line)
    f.close()

def delete_punctuations(doc):
    punctuations = '،:؛؟!»«()[]"*,{.}@!?/'
    edited_doc = doc.translate(str.maketrans('', '', punctuations))
    edited_doc = doc.translate(str.maketrans('', '', string.punctuation))
    
    return edited_doc

def delete_stopWords(doc):
    edited_doc = doc
    for s in stop_words:
        my_regex = r"\b"+s+r"\b"
        edited_doc = re.sub(my_regex , "", edited_doc)
    return edited_doc

def delete_highFrequencyWords(inverted_indexes):
    df_temp = df.copy()
    for i in inverted_indexes:
        if len(i[1])/len(df_temp) > 0.8:
            inverted_indexes.remove(i)
    return inverted_indexes

def delete_postfixes(doc):
    edited_doc = doc
    for p in postfixes:
        my_regex = p + r"\b"
        edited_doc = re.sub(my_regex , "", edited_doc)
    return edited_doc

def delete_prefixes(doc):
    edited_doc = doc
    for p in postfixes:
        my_regex = r"\b" + p
        edited_doc = re.sub(my_regex , "", edited_doc)
    return edited_doc

def replaceWithRoot(tokens):   
    for i in range(0, len(tokens)):
        for root in verb_roots:
            if search(root[0], tokens[i]) or search(root[1], tokens[i]):
                common = False
                for c in common_words:
                    if c == tokens[i]:
                        common = True
                        # print("common")
                        break
                if common == False:
                    # print("root")
                    tokens[i] = tokens[i].replace(tokens[i], root[2])

    return tokens

def replaceArabicWords(doc):
    doc = doc.replace('ك', 'ک')
    doc = doc.replace('ئ', 'ی')
    doc = doc.replace('ي', 'ی')
    doc = doc.replace('ؤ', 'و')
    doc = doc.replace('هٔ', 'ه')
    doc = doc.replace('ة', 'ه')
    doc = doc.replace('آ', 'ا')
    doc = doc.replace('أ', 'ا')
    doc = doc.replace('إ', 'ا')
    return doc

def pluralToSingular(tokens):
    for i in range(0, len(tokens)):
        for ps in plural_singular:
            if search(ps[0], tokens[i]):
                tokens[i] = tokens[i].replace(tokens[i], ps[1])
    return tokens

def tokenize(df):
    content = df.content

    tokens = []
    for i in range(0, content.size):
        doc = content[i]
        # 68000 tokens
        doc = delete_punctuations(doc)
        # 50000 tokens
        doc = delete_stopWords(doc)

        doc = replaceArabicWords(doc)

        doc = delete_postfixes(doc)

        doc = delete_prefixes(doc)
        
        tokenized_doc = doc.split()
        tokenized_doc = replaceWithRoot(tokenized_doc)
        tokenized_doc = pluralToSingular(tokenized_doc)

        for token in tokenized_doc:
            temp = []
            temp.append(token)
            temp.append(df.id[i])
            tokens.append(temp)
    tokens.sort()
    return tokens

def create_inverted_indexes(tokens):
    inverted_indexes = []
    doc_temp = []
    token_temp = ""
    for token in tokens:
        if token[0] == token_temp:
            doc_temp.append(token[1])
        else:
            temp = []
            temp.append(token_temp)

            (unique, counts) = np.unique(doc_temp, return_counts=True)
            frequencies = np.asarray((unique, counts)).T
            
            temp.append(frequencies)
            inverted_indexes.append(temp)
            doc_temp = []

        token_temp = token[0]
        # if len(doc_temp) > 0 and token[1] != doc_temp[-1]:
        doc_temp.append(token[1])
    
    inverted_indexes = delete_highFrequencyWords(inverted_indexes)
    return inverted_indexes

def search_token(inverted_indexes, token_name):
    for token in inverted_indexes:
        if token[0] == token_name:
            return token[1]
    print("this token not exist in our database")

def print_res(df, res):
    p_res = pd.DataFrame()

    p_res = pd.merge(df, res, on=['id'], how='inner')
    our_res = p_res[['id', 'rank', 'url']]
    our_res = our_res.sort_values(["rank"], ascending=False)

    print(our_res)
    

def query_processing(df, inverted_indexes, query):
    res = []
    docs_id = pd.DataFrame()


    query = delete_punctuations(query)
    query = delete_stopWords(query)
    query = delete_postfixes(query)
    query = delete_prefixes(query)
    query = replaceArabicWords(query)

    tokenized_query = query.split()
    tokenized_query = replaceWithRoot(tokenized_query)
    tokenized_query = pluralToSingular(tokenized_query)

    if len(tokenized_query) == 0:
        print("It looks like there aren't many great matches for your search")
        return


    for i in range (0 ,len(tokenized_query)):
        temp_docs_id = pd.DataFrame(search_token(inverted_indexes, tokenized_query[i]))
        docs_id = pd.DataFrame(docs_id.append(temp_docs_id))


    docs_id = docs_id[0].value_counts().reset_index()
    docs_id.columns = ['id', 'rank']
    print_res(df, docs_id)





def ranked_search(df, inverted_indexes, query):
    res = []
    docs_id = pd.DataFrame()


    query = delete_punctuations(query)
    query = delete_stopWords(query)
    query = delete_postfixes(query)
    query = delete_prefixes(query)
    query = replaceArabicWords(query)

    tokenized_query = query.split()
    tokenized_query = replaceWithRoot(tokenized_query)
    tokenized_query = pluralToSingular(tokenized_query)

    if len(tokenized_query) == 0:
        print("It looks like there aren't many great matches for your search")
        return

    (unique, counts) = np.unique(tokenized_query, return_counts=True)
    tokenized_query = np.asarray((unique, counts)).T
    print(tokenized_query)

    doc_score = []
    for i in range (0 ,len(tokenized_query)):
        tf_query = 1 + math.log(int(tokenized_query[i][1]), 10)
        print(tokenized_query[i][0])
        docs = search_token(inverted_indexes, tokenized_query[i][0])
        print(docs)
        print(len(df)/len(docs))
        idf = math.log(len(df)/len(docs), 10)
        print(idf)
        w_termInQuery = tf_query*idf
        print("weight term in q: ", tf_query, idf, w_termInQuery)

        for i in range(len(docs)):
            temp = []
            temp.append(docs[i][0])
            tf_doc = 1 + math.log(int(docs[i][1]), 10)
            w_doc = tf_doc*idf
            temp.append(w_termInQuery*w_doc)
            doc_score.append(temp)

    doc_score.sort()
    doc_score = pd.DataFrame(doc_score)
    doc_score.columns = ['doc', 'score']
    df_doc_score = doc_score.groupby(["doc"]).score.sum().reset_index()
    print(df_doc_score)




    #     # temp_docs_id = pd.DataFrame(search_token(inverted_indexes, tokenized_query[i]))
    #     # docs_id = pd.DataFrame(docs_id.append(temp_docs_id))


    # docs_id = docs_id[0].value_counts().reset_index()
    # docs_id.columns = ['id', 'rank']
    # print_res(df, docs_id)
    

    


In [ ]:
df = read_dataset("datasets/", "IR_Spring2021_ph12_7k.xlsx")
read_files()

In [ ]:
tokens = tokenize(df)

In [ ]:
inverted_indexes = create_inverted_indexes(tokens)

In [197]:
print(len(tokens))
print(len(inverted_indexes))

for i in range(5000, 5100):
    print(inverted_indexes[i][0])
    print(inverted_indexes[i][1])

  2]
 [5056    2]
 [5057    2]
 [5058    2]
 [5061    2]
 [5079    2]
 [5136   10]
 [5148    2]
 [5165    2]
 [5234    2]
 [5242    2]
 [5256    2]
 [5264    2]
 [5273    2]
 [5297    4]
 [5350    2]
 [5357    2]
 [5411    2]
 [5424    4]
 [5426    2]
 [5456    2]
 [5466    2]
 [5514   12]
 [5557    2]
 [5580    2]
 [5605    2]
 [5609    2]
 [5641    2]
 [5643    2]
 [5650    2]
 [5668    2]
 [5688    2]
 [5691    2]
 [5743    2]
 [5751    2]
 [5752    2]
 [5753    4]
 [5778    2]
 [5779    2]
 [5824    2]
 [5830    2]
 [5831    2]
 [5834    2]
 [5835    2]
 [5844    4]
 [5865    2]
 [5873    2]
 [5884    2]
 [5894    2]
 [5899    2]
 [5903    4]
 [5930    2]
 [5966    2]
 [5971    2]
 [5998    2]
 [6003    2]
 [6015    2]
 [6022    2]
 [6032    2]
 [6064    2]
 [6109    2]
 [6123    2]
 [6135    4]
 [6143    4]
 [6164    8]
 [6216    2]
 [6226    6]
 [6251    2]
 [6257    2]
 [6260    2]
 [6265    2]
 [6301    2]
 [6306    2]
 [6323    2]
 [6514    2]
 [6515    2]
 [6521    2]
 [6529 

In [126]:
print(search_token(inverted_indexes, "شد"))

this token not exist in our database
None


In [162]:
query = "علی حسین"
# query_processing(df, inverted_indexes, query)

ranked_search(df, inverted_indexes, query)


[['حسین' '1']
 ['علی' '1']]
حسین
[[  20    3]
 [  30    2]
 [  31    2]
 [  33    4]
 [  37    4]
 [  67    2]
 [  69    2]
 [  71    2]
 [ 107    2]
 [ 185    2]
 [ 193    2]
 [ 205    2]
 [ 206    2]
 [ 213    2]
 [ 218    2]
 [ 219    2]
 [ 236    2]
 [ 262    2]
 [ 265    2]
 [ 267    2]
 [ 290    2]
 [ 314    2]
 [ 341    2]
 [ 358    4]
 [ 377    6]
 [ 383    2]
 [ 386    2]
 [ 412    2]
 [ 421    4]
 [ 433    2]
 [ 467    2]
 [ 477    2]
 [ 481    2]
 [ 490    4]
 [ 491    2]
 [ 507    2]
 [ 514    2]
 [ 518    2]
 [ 520    2]
 [ 523    6]
 [ 526    6]
 [ 529    2]
 [ 536    2]
 [ 537   14]
 [ 540    2]
 [ 541    8]
 [ 544   22]
 [ 571    8]
 [ 595    2]
 [ 602    6]
 [ 608    2]
 [ 624    4]
 [ 641    2]
 [ 657    2]
 [ 659   28]
 [ 666    2]
 [ 692    2]
 [ 743    6]
 [ 749   14]
 [ 752   38]
 [ 763    2]
 [ 764    4]
 [ 769    2]
 [ 771    2]
 [ 773    2]
 [ 774    2]
 [ 776    2]
 [ 777    2]
 [ 779    2]
 [ 812    2]
 [ 813    4]
 [ 824    2]
 [ 829    8]
 [ 834    2]
 [ 84

In [ ]:
tokens[2000]

In [ ]:
create_inverted_indexes(tokens)

In [ ]:
b = {'a','a','b'}
print(b.count('a'))

In [ ]:
docs = search_token(inverted_indexes, "مسیح")
print(docs)
idf = math.log(len(df)/len(docs), 10)
print(idf)



In [196]:

li1 = [['6', 1], ['9', 4], ['3', 14], ['8', 12]]
# heapq._heapify_max(li1[0])
# print(li1)

hp = []
for e in li1:
    hp.append((e[1], e[0]))

nlargest = heapq.nlargest(3, hp)
k_high_ranked = []
for i in nlargest:
    k_high_ranked.append([i[1], i[0]])

print(k_high_ranked)


[['3', 14], ['8', 12], ['9', 4]]


In [228]:
def calculate_tfidf(df, inverted_indexes):
    weighted_inverted_index = inverted_indexes.copy()
    len_df = len(df)
    for i in range(3):
        if len(weighted_inverted_index[i][1]) > 0:
            print(weighted_inverted_index[i])
            # print(len(inverted_indexes[i][1]))
            idf = math.log(len_df/len(weighted_inverted_index[i][1]), 10)
            print("idf = ", idf)
            for j in range(len(weighted_inverted_index[i][1])):
                tf = 1 + math.log(int(weighted_inverted_index[i][1][j][1]), 10)
                print("tf = ", tf)
                w = tf*idf
                print("w : ",w)
                weighted_inverted_index[i][1][j][1] = w
    print(weighted_inverted_index[0])
    print(weighted_inverted_index[1])



In [227]:
calculate_tfidf(df, inverted_indexes)

['0054', array([[6768,    3],
       [6774,    4],
       [6780,    4]], dtype=int64)]
idf =  3.3679767852945943
tf =  1.4771212547196624
w :  4.9749100949610465
tf =  1.6020599913279623
w :  5.395700859441836
tf =  1.6020599913279623
w :  5.395700859441836
['011', array([[4324,    3]], dtype=int64)]
idf =  3.8450980400142565
tf =  1.4771212547196624
w :  5.6796760413859735
['', array([], shape=(0, 2), dtype=float64)]
['0054', array([[6768,    4],
       [6774,    5],
       [6780,    5]], dtype=int64)]


In [222]:
weighted_inverted_index = inverted_indexes.copy()
print(weighted_inverted_index[0])

['', array([], shape=(0, 2), dtype=float64)]


In [233]:
print(inverted_indexes[1])

['0054', array([[6768,    5],
       [6774,    5],
       [6780,    5]], dtype=int64)]
